Save AN data from a model file run to an NWB formatted file
-----------------------------------------------------------
In this script, we read a file generated by model_run with the "--saveall"
flag set so that all section voltages (middle of the section) are stored in the file.
The first set of code blocks does this and lets us read and plot the data.


In [12]:
"""
Necessary imports
Note: plot in separate window - lets us use the tools
"""
%matplotlib qt  
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as mpl
from pathlib import Path
import numpy as np
import pandas as pd
import pynwb as NWB


In [14]:
"""
You cannot run this unless you have the model data output file
"""

fn = Path('VCN_c06/Simulations/IV/VCN_c06_pulse_XM13_nacncoop_II_monitor.p')
fn = Path('VCN_c06/Simulations/IV/VCN_c06_pulse_XM13_nacncoop_II_monitor_noDendNa_Full.p')
fn = Path('VCN_c11/Simulations/AN/AN_Result_VCN_c11_inp=self_XM13_nacncoop_II_soma=1.203_dend=1.510_ASA=1.203_delays_multisite_001_tonepip_050dB_4000.0_HS.p.dat')
basepath = Path('/Users/pbmanis/Desktop/Python/VCN-SBEM-Data/VCN_Cells')
fnb = Path(basepath, fn)
print(fnb.is_file())

d = pd.read_pickle(fnb)  # reads the original data file

True


In [15]:
"""
Make params dict into a simple text list using toml
"""
import toml
params = toml.dumps(d['Params'])  # convert to a string
print(params)
# print('trial0 keys: ', d['trials'][0].keys())
# print(toml.dumps(d['trials'][0]['stimInfo']))
# toml.dumps(d['Params'])

cell = "VCN_c11"
AMPAScale = 1.0
ANSynapseType = "multisite"
ANSynapticDepression = 0
initIVStateFile = "None"
initANStateFile = "/Users/pbmanis/Desktop/Python/VCN-SBEM-Data/VCN_Cells/VCN_c11/Initialization/ANneuronState_XM13_nacncoop_II_soma=1.203_dend=1.510_ASA=1.203_inp=self_multisite.dat"
simulationFilename = "/Users/pbmanis/Desktop/Python/VCN-SBEM-Data/VCN_Cells/VCN_c11/Simulations/AN/AN_Result_VCN_c11_inp=self_XM13_nacncoop_II_soma=1.203_dend=1.510_ASA=1.203_all_multisite_001_tonepip_050dB_4000.0_HS.p.dat"
hocfile = "VCN_c11.hoc"
fullhocfile = false
usedefaulthoc = true
cellType = "Bushy"
modelName = "XM13_nacncoop"
modelType = "II"
SGCmodelType = "cochlea"
species = "mouse"
Ra = 150.0
dt = 0.025
soma_inflation = "1.2028436195935286"
soma_autoinflate = true
dendrite_inflation = "1.5096536427887555"
dendrite_autoinflate = true
dendrite_fromsoma = false
ASA_inflation = "1.2028436195935286"
ASA_fromsoma = true
lambdaFreq = 2000.0
dendriteelectrode = "Proximal_Dendrite"
sequence = ""

In [16]:
"""
Plot all locations (for reference) for one stimulus (AN data)
Each trial contains a dict:
dict_keys(['stimInfo', 'spikeTimes', 'inputSpikeTimes', 'time',
'somaVoltage', 'dendriteVoltage', 'allDendriteVoltages', 
'stimWaveform', 'stimTimebase'])

"""

f, ax = mpl.subplots(1,1)
spines_to_remove = ['top', 'right']
ax = [ax]
for a in ax:
    for spine in spines_to_remove:
        a.spines[spine].set_visible(False)
# for trial in d['trials']:
trial = 0  # for now, just handle one trial
dx = d['trials'][trial]
print('trial: ', trial)
print(dx.keys())  # for reference
time = dx['time']
data_array = None
for tx in dx['allDendriteVoltages']:
    for v in dx['allDendriteVoltages'][tx]:
        y = dx['allDendriteVoltages'][tx][v]
        if data_array is None:
            data_array = np.zeros((len(dx['allDendriteVoltages'][tx]), len(y)))
        secno = int(v[9:-1])  # get the section number
        data_array[secno,:] = y  # store data in section order

for i in range(data_array.shape[0]):
    ax[0].plot(time, data_array[i,:], linewidth=0.25)
 
mpl.show()

trial:  0
dict_keys(['stimInfo', 'spikeTimes', 'inputSpikeTimes', 'time', 'somaVoltage', 'dendriteVoltage', 'allDendriteVoltages', 'stimWaveform', 'stimTimebase'])


In [17]:
"""
Assemble the spike times
"""

ANSpikeTimes = []  # store as dict so that input number is an explicit key
for i in range(len(dx['inputSpikeTimes'])):
    ANSpikeTimes.append(dx['inputSpikeTimes'][i])


Now save the data 

In [19]:
"""
Save AN data to NWB formatted file
"""
import datetime as datetime
from dateutil.tz import tzlocal
import h5py

outfilename = 'test'
datafilename = fn
sessionno = 0
subject = NWB.file.Subject(age="0", 
        description='vcnmodel', genotype='None', sex='None', species='Computer', 
        subject_id="1", weight='None')

#     print(info)
nwbfile = NWB.NWBFile('AN Data set', str(datafilename), datetime.datetime.now(tzlocal()),
              experimenter='Manis, Paul',
              lab='Manis Lab',
              institution='UNC Chapel Hill',
              experiment_description='Model Output',
              session_id=f"{sessionno:d}",
              notes=params,
              subject=subject
                     )

device = NWB.device.Device('cnmodel')
nwbfile.add_device(device)
# print(an_timestamps)
ANSpikeOrigin = []
for i in range(len(ANSpikeTimes)):
    ANSpikeOrigin.append(i*np.ones(len(ANSpikeTimes[i])))
an_spike_origin = [y for x in ANSpikeOrigin for y in x]
an_time_stamps = [y for x in ANSpikeTimes for y in x]
# print(len(an_timestamps))
# print(an_spike_origin)
aninputs = NWB.base.TimeSeries(name="ANSpikeTimes",
                               data=an_spike_origin,
                               unit='ms',
                               comments="Input AN spike trains", 
                               description= "AN Spike Trains; data is stored in TimeStamps as a dictionary",
                               timestamps=an_time_stamps)
nwbfile.add_stimulus(aninputs)

elec = NWB.icephys.IntracellularElectrode(name="elec0",
                               description='electrodes in middle of section',
                               device=device)
nwbfile.add_ic_electrode(elec)  # not well documented!

istim = NWB.icephys.CurrentClampStimulusSeries(name="Ics", data=np.array(dx['stimWaveform']),
        starting_time=0., rate=d['Params']['dt'],
        electrode=elec, gain=1.0, sweep_number=1)
vdata = NWB.icephys.CurrentClampSeries('CCData', data=data_array, 
        unit='volts', electrode=elec, gain=1.0, bias_current=0.0, bridge_balance=0., 
        capacitance_compensation=0., stimulus_description='NA', 
        resolution=0.0, conversion=1.0, timestamps=None, 
        starting_time=0., rate=d['Params']['dt'], 
        comments='no comments', description='no description', 
        control=None, control_description=None, sweep_number=1)
nwbfile.add_acquisition(istim)
nwbfile.add_acquisition(vdata)

with NWB.NWBHDF5IO(str(Path(outfilename))+'.nwb', 'w') as io:
    io.write(nwbfile)

print('ok')

ok


The block below reads and displays the data from the NWB file
-------------------------------------------------------------
Provided as an example of how to read the data, and also
as confirmation that the data contains what is expected.


In [27]:
"""
Read the nwb file back in and display the data
Note that the [()] construct is essential for accessing the data

"""

infile = Path(outfilename).with_suffix(".nwb")
if infile.is_file():
    try:
        io = NWB.NWBHDF5IO(str(infile), 'r')
        nwbdata = io.read()

        notes = nwbdata.notes
        vcs = nwbdata.acquisition['CCData']
        data = vcs.data[()]
        ANSpikes = nwbdata.stimulus['ANSpikeTimes']
        anspikeorigin = ANSpikes.data[()]
        anspiketimes = ANSpikes.timestamps[()]
    except:
        print('Error reading data file')
    finally:
        io.close()
 
print(notes)  # holds most of the run parameters in a text string.

timebase = np.arange(0, vcs.rate*data.shape[1], vcs.rate)

f, axl = mpl.subplots(2,1, sharex=True)
for i in range(data.shape[0]):  # plot traces for every section
    axl[0].plot(timebase, data[i,:], linewidth=0.35)

ost = np.argsort(anspiketimes)  # ordered sort times
anspikeorigin = anspikeorigin[ost]
anspiketimes = anspiketimes[ost]
ninputs = np.max(anspikeorigin)
coincidence_window = 0.05  # adjust this to change the coincidence window

intervals = np.where(np.diff(anspiketimes) < coincidence_window)[0]

# plot AN spikes that occur together within a close coincidence window (like Joris and Smith)
axl[1].scatter(anspiketimes[intervals], anspikeorigin[intervals], 
            c='r', s=25, marker='|', linewidth=2, facecolor='r', edgecolor='r')
axl[1].scatter(anspiketimes[intervals+1], anspikeorigin[intervals+1], 
            c='r', s=25, marker='|', linewidth=2, facecolor='r', edgecolor='r')

noncoinc = [i for i, x in enumerate(anspiketimes) if x not in intervals and x not in intervals+1] 

axl[1].scatter(noncoinc, anspikeorigin[noncoinc], 
            c='k', s=25, marker='|', linewidth=1, facecolor='k', edgecolor='None')

xlim = axl[0].get_xlim()
axl[1].set_xlim(xlim)
axl[0].set_ylabel('V(mv)', fontsize=9)
axl[1].set_ylabel('Input #', fontsize=9)
axl[1].set_xlabel('T (ms)')
axl[0].set_title('All section voltages')
axl[1].set_title(f"Input Spikes and Coincidences (dt={coincidence_window:.2f} ms)")

for ax in axl:
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

mpl.show()

cell = "VCN_c11"
AMPAScale = 1.0
ANSynapseType = "multisite"
ANSynapticDepression = 0
initIVStateFile = "None"
initANStateFile = "/Users/pbmanis/Desktop/Python/VCN-SBEM-Data/VCN_Cells/VCN_c11/Initialization/ANneuronState_XM13_nacncoop_II_soma=1.203_dend=1.510_ASA=1.203_inp=self_multisite.dat"
simulationFilename = "/Users/pbmanis/Desktop/Python/VCN-SBEM-Data/VCN_Cells/VCN_c11/Simulations/AN/AN_Result_VCN_c11_inp=self_XM13_nacncoop_II_soma=1.203_dend=1.510_ASA=1.203_all_multisite_001_tonepip_050dB_4000.0_HS.p.dat"
hocfile = "VCN_c11.hoc"
fullhocfile = false
usedefaulthoc = true
cellType = "Bushy"
modelName = "XM13_nacncoop"
modelType = "II"
SGCmodelType = "cochlea"
species = "mouse"
Ra = 150.0
dt = 0.025
soma_inflation = "1.2028436195935286"
soma_autoinflate = true
dendrite_inflation = "1.5096536427887555"
dendrite_autoinflate = true
dendrite_fromsoma = false
ASA_inflation = "1.2028436195935286"
ASA_fromsoma = true
lambdaFreq = 2000.0
dendriteelectrode = "Proximal_Dendrite"
sequence = ""